# Логистическая регрессия

Обучим логистическую регрессию для предсказания того, откликнется клиент  
на рекламное предложение (target = 1) или нет (target = 0).

## Подлючение библиотек, загрузка и обзор данных

In [505]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [506]:
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/clients_data.csv')

In [507]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


In [508]:
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)

## Практика

**Задание**  
Выведите на экран количество объектов каждого класса. Сколько процентов объектов относятся к положительному классу?  
Ответ округлите до целого числа (например, если доля объектов положительного класса равна 0.412, в ответ запишите 41,  
имея в виду 41 процент).

In [509]:
print(f'Доля объектов положительного класса {round((data["TARGET"].value_counts()[1]/data.shape[0])*100)} %')

Доля объектов положительного класса 12 %


Обучим логистическую регрессию с параметрами по умолчанию.

In [510]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

In [511]:
lr.fit(Xtrain, ytrain)

prediction = lr.predict(Xtest)

Метрику accuracy не стоит использовать при сильном дисбалансе классов. Поэтому посчитайте f1_score для оценки качества модели на тестовых данных.

f1_score принимает значения от 0 до 1. Чем ближе к 1, тем лучше модель.

In [512]:
from sklearn.metrics import f1_score
f1_score(ytest, prediction)

0.0

**Вопрос**  
Чему равен `f1_score`?

**Ответ**
0

Удивительно, да?

Давайте разберемся, почему качество такое низкое.

Предскажем вероятности классов с помощью обученной логистической регрессии на тестовых данных.

In [513]:
probs_test = lr.predict_proba(Xtest)

probs_test[:10]

array([[0.86331972, 0.13668028],
       [0.86305957, 0.13694043],
       [0.80967732, 0.19032268],
       [0.73110167, 0.26889833],
       [0.87780414, 0.12219586],
       [0.96122864, 0.03877136],
       [0.95965491, 0.04034509],
       [0.96560659, 0.03439341],
       [0.92114871, 0.07885129],
       [0.9349701 , 0.0650299 ]])

По вероятностям видно, что вероятности отнесения к положительному классу очень низкие.

Попробуем изменить порог для перевода вероятности в классы.

**Вопрос**  
Чему равен `f1_score`, если все объекты с вероятностью не меньшей 0.1, относить к положительному классу?

Ответ округлите до сотых.

In [514]:
ypred = probs_test[ : , 1] >= 0.1
print(f'f1 score = {f1_score(ytest, ypred).round(2)}')

f1 score = 0.25


У обученной модели можно посмотреть веса (как и в линейной регрессии).  
Выведем на экран веса модели (`model.coef_`, `model.intercept_`).

In [515]:
lr.coef_, lr.intercept_

(array([[-5.55214705e-02, -7.90868782e-04, -4.28172545e-04,
         -1.07248154e-03, -1.34958177e-03, -4.95807875e-04,
          1.13131722e-05, -1.80011616e-03, -1.26843809e-03,
         -1.45283567e-05]]),
 array([-0.00115086]))

Создайте `pd.DataFrame`, где в первом столбце стоят названия признаков, а во втором - их веса (так удобнее анализировать результат).  
Отсортируйте таблицу по убыванию весов.

**Вопрос**  
Какой признак имеет наибольший положительный вес?

In [516]:
pd.DataFrame(lr.coef_[0], index=X.columns, columns= ['coef']).sort_values('coef', ascending=False)

,coef
PERSONAL_INCOME,0.000011
LOAN_DLQ_NUM,-0.000015
SOCSTATUS_PENS_FL,-0.000428
DEPENDANTS,-0.000496
SOCSTATUS_WORK_FL,-0.000791
GENDER,-0.001072
LOAN_NUM_CLOSED,-0.001268
CHILD_TOTAL,-0.001350
LOAN_NUM_TOTAL,-0.001800
AGE,-0.055521


**Ответ** PERSONAL_INCOME

## Бонус

**Задание 1**  
Подберите порог для перевода вероятностей в классы, дающий максимальное значение `f1_score`.

In [517]:
dict_ = {}

for i in np.arange(0,0.5, 0.01):
    key = i
    y_pred_2 = np.where(probs_test[ : , 1] > i, 1, 0)
    dict_[key] = f1_score(ytest, y_pred_2).round(4)

max_key = max(dict_, key=dict_.get)
max_value = dict_[max_key]

print(f"max_prob: {max_key}, max_f1_score: {max_value}")

max_prob: 0.1, max_f1_score: 0.247


**Задание 2**  
Во вложенном цикле подберите одновременно коэффициент регуляризации `C` у логистической регрессии и порог для перевода вероятностей в классы, дающие максимальное значение `f1_score`.

По-хорошему, чтобы не переобучиться, эти величины надо подбирать не по тесту, а по отдельной выборке.

Поэтому разобъем данные изначально на три части: `Xtrain`, `Xval`, `Xtest`.

*   В цикле при подборе `С` и порога будем обучаться по `Xtrain`, а предсказывать и измерять качество по `Xval`.

*   Качество итоговой модели с найденными `C` и порогом измерьте по `Xtest`.

Так не переобучимся!


In [518]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

In [519]:
best_f = 0
best_p = 0
best_c = 0

for c in [0.001, 0.01, 1, 10, 100]:
       for p in np.arange(0, 0.5, 0.05):
        model = LogisticRegression(C = c)
        model.fit(Xtrain_new, ytrain_new)
        ypred_val = model.predict_proba(Xval)[:,1] >= p
        f1 = f1_score(yval, ypred_val)
        if f1 > best_f:
            best_f = f1
            best_p = p
            best_c = c


print(best_c, best_p, best_f.round(2))

0.01 0.1 0.23


In [520]:
best_f = 0
best_p = 0

model = LogisticRegression()
model.fit(Xtrain_new, ytrain_new)

for p in np.arange(0, 0.5, 0.05):
        ypred = model.predict_proba(Xtest)[:,1] >= p
        f1 = f1_score(ytest, ypred)
        if f1 > best_f:
            best_f = f
            best_p = p
print(f'Максимальное значение f1 score {best_f.round(2)} достигается при p = {best_p.round(3)}')      

Максимальное значение f1 score 0.25 достигается при p = 0.0
